## Предобработка текстовых данных

Попробуем предобработать текстовые данные из датасета Airbnb, чтобы улучшить качество работы Vowpal Wabbit.

In [1]:
import pandas as pd
import numpy as np

import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [2]:
df = pd.read_csv("airbnb-100k.tsv", delimiter='\t')
df.sample(5)

,Price,Description
15864,NaN,"Situated at the Western part ocentral Copenhagen. Overviewing meatpacking district. 5 min walk from town hall square and central station. Its an egoist appartment with two master bedrooms but can be made for 6 persons. (PHONE NUMBER HIDDEN)esterbro is located just outside Copenhagen’s city center—the Inner City or Indre By—making it a very attractive place to live, as are the other areas immediately outside the center: the Indre Nørrebro (“Inner Nørrebro”), Indre Østerbro (“Inner Østerbro”), Frederiksberg, and Christianshavn. The district is located west of the city center at the location of the old Western Gate (‘’Vesterport’’), access way into the old city. The gate, along with the other three gates into the old city-- Østerport (""Eastern Gate"") near the current Østerport station), N..."
9779,50.0,"Dans le 17ème arrondissement de Paris, beau studio meublé très clair et calme, au 3ème étage sans ascenseur, idéal pour visiter Paris! Description: Environ 25m2, poutres apparentes,1 chambre et une cuisine/salle à manger. Salle de douche avec WC"
49079,90.0,"A clean and cosy one bedroom apartment in a georgian house on a quiet street for up to 3. Ideal location for exciting east end and London culture, business trips and family visits,great transport links Everything provided for comfort and convenience. Original classic style, relaxing and homely. Wood floor, simple furnishing with modern kitchen fittings and equipment,storage space and pretty yard and woodland garden This is a self contained apartment with separate, independent entrance. Visitors can enjoy the garden and there is a shared laundry room. Safe on street parking available outside the house. I live in the other part of the house and am available to greet visitors and give them as much or as little advice,help or friendly interaction that they need. I have lived in Hackney ..."
11920,25.0,"Amplia y cómoda habitación en piso de lujo en una de las mejores zonas de Madrid. Barrio tranquilo y bien comunicado, cercano al metro y múltiples paradas de autobús Línea 4 de metro Estación de Arturo Soria, líneas (PHONE NUMBER HIDDEN) de buses"
95815,200.0,"A beautifully modern and well presented family home in the popular district of Canonmills. Suitable for a small group of travellers and guests travelling with young children. Short walking distance to City Centre, Supermarkets and Bus / Tram stops. Property has lift access to terrace. Apartment is located on the terrace level. - Bright and spacious open plan kitchen / dining area / living room with sofa bed. (Sofa bed suitable for 1 adult or 2 children only.) - Private terrace with BBQ and outdoor seating. (Charcoal for BBQ will be provided.) - Double bedroom with en suite (shower). - Smaller room with two single beds (can convert into a double). - Family bathroom (shower and bath). - Free car parking (1 car only) Guests will have access to the whole property. - All appliances can b..."


Наши шаги следующие:

- разобъем строки на токены (слова)
- приведем все в нижний регистр
- уберем стоп-слова (частоупотребимые члова, не несущие смысла: для английского языка это артикли, местоимения, модальные глаголы и т.д.)
- проведем стемминг (выделим псевдооснову слова)

В библиотеке NLTK (Natural Language Toolkit) реализованы методы для работы с естественным языком, в том числе, есть наборы стоп-слов для разных языков (включая русский).

In [3]:
import re
import nltk.data 
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

Строго говоря, токенизацию тоже можно сделать методами из NLTK, но пока мы оставим простую токенизацию с помощью регулярных выражений.

In [4]:
# токенизация 
words_only = re.compile('[a-z]+')

def letters(s, regex = words_only):
    if isinstance(s, str):
        return words_only.findall(s.lower())
    else:
        return []

# удаление стоп-слов
def remove_stopwords(tokens, sw = stop_words):
    return [t for t in tokens if not t in sw]

def preprocess(s):
    return remove_stopwords(letters(s))

In [5]:
s = 'I get help from a local service called "Key Butler" to welcome you and let you in etc, and they....' 

preprocess(s)

['get',
 'help',
 'local',
 'service',
 'called',
 'key',
 'butler',
 'welcome',
 'let',
 'etc']

In [8]:
df['clean_description'] = df.Description.map(preprocess)
df.sample(4)

,Price,Description,clean_description
43536,135.0,"These stunning 2 bedroom apartments are in the original living quarters of the house. They provide spacious accommodation with a large central drawing room and an open-plan kitchen. A separate master bedroom and another smaller bedroom look onto picturesque Doughty Street. Stunning Two Bedroom, one bathroom apartment in No-5 Doughty Street, Bloomsbury, London Stunning Two Bedroom, one bathroom apartment in No-5 Doughty Street, Bloomsbury, London These stunning 2 bedroom apartments are in the original living quarters of the house. They provide spacious accommodation with a large central drawing room and an open-plan kitchen. A separate master bedroom and another smaller bedroom look into the pictures Doughty Street. This attractive apartment of 52m2 disposes a spacious living room wit...","[stunning, bedroom, apartments, original, living, quarters, house, provide, spacious, accommodation, large, central, drawing, room, open, plan, kitchen, separate, master, bedroom, another, smaller, bedroom, look, onto, picturesque, doughty, street, stunning, two, bedroom, one, bathroom, apartment, doughty, street, bloomsbury, london, stunning, two, bedroom, one, bathroom, apartment, doughty, street, bloomsbury, london, stunning, bedroom, apartments, original, living, quarters, house, provide, spacious, accommodation, large, central, drawing, room, open, plan, kitchen, separate, master, bedroom, another, smaller, bedroom, look, pictures, doughty, street, attractive, apartment, disposes, spacious, living, room, sofa, bed, flat, screen, tv, pay, per, view, channels, free, wifi, internet, ..."
63320,70.0,"Nice apartment situated in Puerto Alcudia. Really calm street,but close to the restaurants, supermarkets, marina and the ba","[nice, apartment, situated, puerto, alcudia, really, calm, street, close, restaurants, supermarkets, marina, ba]"
50157,100.0,"This is a cozy one bedroom apartment with a spacious kitchen, living room and two balconies. The apartment is located in one of the upcoming neighborhoods of Amsterdam- Westerpark, which is a quiet area neighboring the famous district Jordaan. This apartment comes with a cat! So if you like cats and are not allergic to them you are welcome to stay. You are kindly asked to feed and be friendly to him. He mostly stays outside and occasionally sleeps indoors. The apartment is a very cozy place offering a comfortable and relaxing stay especially for those who are not only looking to have fun in Amsterdam but also to relax. The bedroom features a comfy two-person bed with windows facing the backyard meaning you get a good nights sleep! The bathroom featuring a sink and a bathtub (!) adjoins...","[cozy, one, bedroom, apartment, spacious, kitchen, living, room, two, balconies, apartment, located, one, upcoming, neighborhoods, amsterdam, westerpark, quiet, area, neighboring, famous, district, jordaan, apartment, comes, cat, like, cats, allergic, welcome, stay, kindly, asked, feed, friendly, mostly, stays, outside, occasionally, sleeps, indoors, apartment, cozy, place, offering, comfortable, relaxing, stay, especially, looking, fun, amsterdam, also, relax, bedroom, features, comfy, two, person, bed, windows, facing, backyard, meaning, get, good, nights, sleep, bathroom, featuring, sink, bathtub, adjoins, bedroom, relax, nice, bubble, bath, reminiscing, van, gogh, paintings, seen, today, living, room, area, shares, space, fully, equipped, kitchen]"
68364,697.0,"The apartment is located in Nørrebro, right on the boarder of Frederiksberg. The street is quite lively with restaurants, cafés and amazing beer bars just around the corner. The apartment is cosy with everything you'll need for a long or short stay. Our apartment is made up of a fully equipped kitchen, a cosy dining/living room, a bedroom with closet space and a small toilet and bath. It's a great space to relax between trips in Copenhagen or if you want to make dinner and stay in. The whole place is 

Если присмотреться, можно увидеть, что в данных есть тексты не только на английском языке:

In [9]:
df.iloc[41403]

Price                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            350
Description          Le rez-de chaussée de la Maison Duquette est à 2 minutes de marche du métro Villa-Maria, Il s'agit d'une maison familiale avec 4 grandes chambres, salon, sall

Посмотрим, какие еще языки есть.

Воспользуемся библиотекой spaCy, она написана на Cython и в целом довольно быстрая. Официальная документация [тут](https://spacy.io/models). В библиотеке есть предобученные модели для 16 языков, но для русского пока, к сожалению, нет.



In [10]:
import spacy
from spacy_langdetect import LanguageDetector
from tqdm import tqdm_notebook as tqdm
from langdetect import DetectorFactory

DetectorFactory.seed = 0
spacy.util.fix_random_seed(0)

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [11]:
# для ускорения будем определять язык только для первого предложения в тексте

def detect_lang(t):
    if isinstance(t, str):
        text = t.split('.')[0]
        doc = nlp(text)
        return (doc._.language.get('language'))
    else:
        return None

Метод LanguageDetector, определяющий язык текста, не очень быстрый, поэтому воспользуемся библиотекой multiprocessing, чтобы распараллелить процесс:

In [12]:
from multiprocessing import Pool

with Pool(2) as p:
    langs = list(tqdm(p.imap(detect_lang, df.Description), total=len(df)))
    
df['lang'] = langs

In [13]:
df.lang.value_counts()

en         82612
fr          6655
es          3300
it          2756
de          1760
da           525
nl           503
zh-cn        284
af           235
ca           202
ko           164
pt           139
ro           129
UNKNOWN       97
no            77
vi            55
el            51
et            49
tl            44
ru            38
so            33
fi            26
pl            26
hu            23
sw            22
sv            21
id            20
cy            19
lt            16
zh-tw         16
sq            11
tr            10
ja            10
sk            10
sl             5
cs             5
lv             4
hr             3
bg             1
Name: lang, dtype: int64

Последние два шага препроцессинга (удаление стоп-слов, которое мы уже проделали, и стемминг) сильно зависят от языка, поэтому для простоты мы оставим только англоязычные данные, хотя библиотеки NLTK и spaCy умеют работать с большинством европейских языков (для этого нужно лишь скачать соответствующие модели).

In [14]:
eng_df = df[df.lang == 'en']
len(eng_df)

82612

In [15]:
eng_df.to_csv('eng_df.csv', index=False)

In [16]:
del df

Стеммер возьмем из NLTK:

In [17]:
from nltk.stem.snowball import SnowballStemmer

snowball = SnowballStemmer("english")
snowball.stem('apartment')

'apart'

Стемминг - не очень быстрая операция, хотя и более быстрая, чем лемматизация (приведение к нормальной форме, которое чаще используется для языков с богатой морфологией, скажем, русского). Поэтому мы создадим кеш для функции-стеммера, чтобы не делать повторно стемминг для часто встречающихся слов.

In [18]:
from functools import lru_cache

In [19]:
def stemm_description(d):
    @lru_cache(maxsize=128)
    def stemm_token(token):
        return snowball.stem(token)

    return [stemm_token(t) for t in d if len(stemm_token(t)) >= 3]

In [20]:
tokens = eng_df.clean_description[1]

stemm_description(tokens)[:10]

['comfort',
 'calm',
 'apart',
 'room',
 'center',
 'pari',
 'bastill',
 'area',
 'welcom',
 'explain']

In [21]:
from multiprocessing import Pool

with Pool(2) as p:
    stems = list(tqdm(p.imap(stemm_description, eng_df.clean_description), total=len(eng_df)))
    
eng_df['stems'] = stems

In [22]:
eng_df.sample(3)

,Price,Description,clean_description,lang,stems
77476,65.0,"This big bright room is on the second floor of a newly renovated three bedrooms duplex. The central air keeps you cool during summer and warm during winter. There is plenty of living space, a fully equipped kitchen and furnished backyard with a grill. See "" Guest Access "" for more details. The space is equipped with central air and heat. There is a fully equipped chef's kitchen with brand new appliances including a dishwasher. ( See GUEST ACCESS for more information ) Our guest have full access to the chef's kitchen equipped with brand new appliances as well as s fully furnished backyard with charcoal grill. The room has a smart TV hooked up to Netflix, Hulu and Showtime. It is wifi enabled so internet browsing is also an option. There is an iron and iron board. Our guests also hav...","[big, bright, room, second, floor, newly, renovated, three, bedrooms, duplex, central, air, keeps, cool, summer, warm, winter, plenty, living, space, fully, equipped, kitchen, furnished, backyard, grill, see, guest, access, details, space, equipped, central, air, heat, fully, equipped, chef, kitchen, brand, new, appliances, including, dishwasher, see, guest, access, information, guest, full, access, chef, kitchen, equipped, brand, new, appliances, well, fully, furnished, backyard, charcoal, grill, room, smart, tv, hooked, netflix, hulu, showtime, wifi, enabled, internet, browsing, also, option, iron, iron, board, guests, also, access, living, areas, tea, room, comfortable, blue, leather, couch, brown, leather, butterfly, chair, sink, right, perfect, area, enjoy, nice, ...]",en,"[big, bright, room, second, floor, newli, renov, three, bedroom, duplex, central, air, keep, cool, summer, warm, winter, plenti, live, space, fulli, equip, kitchen, furnish, backyard, grill, see, guest, access, detail, space, equip, central, air, heat, fulli, equip, chef, kitchen, brand, new, applianc, includ, dishwash, see, guest, access, inform, guest, full, access, chef, kitchen, equip, brand, new, applianc, well, fulli, furnish, backyard, charcoal, grill, room, smart, hook, netflix, hulu, showtim, wifi, enabl, internet, brows, also, option, iron, iron, board, guest, also, access, live, area, tea, room, comfort, blue, leather, couch, brown, leather, butterfli, chair, sink, right, perfect, area, enjoy, nice, convers, ...]"
90585,57.0,"Walkout apartment with keyless locks. Clean renovated basement with windows. All new furniture and appliances, new queen bed; high quality 100% cotton linens. Unlimited internet and easy access to TTC. Quiet residential neighbourhood with tree-lined streets and Victorian houses between Leslieville and the Danforth. Free parking either behind or close to the house. Parks, waterfront trail, and restaurants close by.","[walkout, apartment, keyless, locks, clean, renovated, basement, windows, new, furniture, appliances, new, queen, bed, high, quality, cotton, linens, unlimited, internet, easy, access, ttc, quiet, residential, neighbourhood, tree, lined, streets, victorian, houses, leslieville, danforth, free, parking, either, behind, close, house, parks, waterfront, trail, restaurants, close]",en,"[walkout, apart, keyless, lock, clean, renov, basement, window, new, furnitur, applianc, new, queen, bed, high, qualiti, cotton, linen, unlimit, internet, easi, access, ttc, quiet, residenti, neighbourhood, tree, line, street, victorian, hous, leslievill, danforth, free, park, either, behind, close, hous, park, waterfront, trail, restaur, close]"
72144,200.0,This is a self contained apartment above my home. A spacious and comfortable backyard space to relax in and easy access to the city. Blocks from Frenchman Street and the quarter with walking access to the Bywater and Marigny,"[self, contained, apartment, home, spacious, comfortable, backyard, space, relax, easy, access, city, blocks, frenchman, street, quarter, walking, access, bywater, marigny]",en,"[self, contain, apart, home, spaci

Сравним качество модели для "сырых" текстов (столбец 'Description'), токенов без стоп-слов (столбец 'clean_description') и стемм (столбец 'stems').

In [23]:
eng_df.dropna(subset=['Price', "stems"], inplace=True)

Y = eng_df['Price']
X_raw = eng_df['Description'].map(letters)
X_tokens = eng_df['clean_description']
X_stems = eng_df['stems']

In [24]:
def convert_to_vw(raw_text, target):
    return "{} |d {}".format(float(target), " ".join(raw_text))

In [25]:
def write_vw(X_data, Y_data, filename):
    with open(filename, "w") as f:
        for x, y in zip(X_data, Y_data):
            vw_object = convert_to_vw(x, y)
            if not vw_object:
                continue
            f.write(vw_object + '\n')

In [26]:
from sklearn.metrics import r2_score

def read_target_from_vw(vw_object):
    return float(vw_object.split(' ')[0])

def calc_r2(predictions_path, answers_path):
    with open(predictions_path, 'r') as f:
        y_pred = np.array([float(value) for value in f.readlines()])
        
    with open(answers_path, 'r') as f:
        y_expected = np.array([read_target_from_vw(value) for value in f.readlines()])
        
    return r2_score(y_expected, y_pred)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, Y, test_size=0.33, random_state=4200)
X_train_tokens, X_test_tokens, y_train, y_test = train_test_split(X_tokens, Y, test_size=0.33, random_state=4200)
X_train_stems, X_test_stems, y_train, y_test = train_test_split(X_stems, Y, test_size=0.33, random_state=4200)

In [29]:
write_vw(X_train_raw, y_train, "airbnb-train-raw.vw")
write_vw(X_test_raw, y_test, "airbnb-test-raw.vw")

write_vw(X_train_tokens, y_train, "airbnb-train-tokens.vw")
write_vw(X_test_tokens, y_test, "airbnb-test-tokens.vw")

write_vw(X_train_stems, y_train, "airbnb-train-stems.vw")
write_vw(X_test_stems, y_test, "airbnb-test-stems.vw")

In [30]:
! head -n 1 airbnb-train-raw.vw

220.0 |d square feet high ceilings king bed completely renovated you won t find many bedroom apartments like this in the west village you re steps away from the meatpacking district high line and whitney museum when you walk outside and you can enjoy this luxurious clutter free airy home when you decide to stay in the apartment is newly renovated including the furniture and finishings some highlights include brand new samsung suhd tv with amazon fire tv high speed wifi kitchen sodastream bialetti espresso maker full set of pots pans bang and olufsen a bluetooth speaker luxurious king size leesa mattress as a guest you will have a key to the main entry of the building as well as the apartment which is a storey walk up aside from a few cupboards and areas of the closet where i will keep my belongings the rest of the apartment is yours to use there is plenty of storage space i will be around manhattan most of the time to help if needed i m always


In [31]:
! head -n 1 airbnb-train-tokens.vw

220.0 |d square feet high ceilings king bed completely renovated find many bedroom apartments like west village steps away meatpacking district high line whitney museum walk outside enjoy luxurious clutter free airy home decide stay apartment newly renovated including furniture finishings highlights include brand new samsung suhd tv amazon fire tv high speed wifi kitchen sodastream bialetti espresso maker full set pots pans bang olufsen bluetooth speaker luxurious king size leesa mattress guest key main entry building well apartment storey walk aside cupboards areas closet keep belongings rest apartment use plenty storage space around manhattan time help needed always


In [32]:
! head -n 1 airbnb-train-stems.vw

220.0 |d squar feet high ceil king bed complet renov find mani bedroom apart like west villag step away meatpack district high line whitney museum walk outsid enjoy luxuri clutter free airi home decid stay apart newli renov includ furnitur finish highlight includ brand new samsung suhd amazon fire high speed wifi kitchen sodastream bialetti espresso maker full set pot pan bang olufsen bluetooth speaker luxuri king size leesa mattress guest key main entri build well apart storey walk asid cupboard area closet keep belong rest apart use plenti storag space around manhattan time help need alway


Теперь обучим модели на всех трех датасетах и сравним результаты:

In [33]:
%%time

! vw --final_regressor airbnb-lin-model-raw.vw.bin airbnb-train-raw.vw --passes 20 -l 3 -c -k  --bit_precision 18

final_regressor = airbnb-lin-model-raw.vw.bin
Num weight bits = 18
learning rate = 3
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airbnb-train-raw.vw.cache
Reading datafile = airbnb-train-raw.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
48400.000000 48400.000000            1            1.0 220.0000   0.0000      174
125191.367188 201982.734375            2            2.0 450.0000   0.5751        9
72780.285416 20369.203644            4            4.0 240.0000  40.3795      170
36699.550247 618.815079            8            8.0  40.0000  10.0007       33
80979.453055 125259.355862           16           16.0 135.0000  52.5366      153
51445.165040 21910.877026           32           32.0  79.0000  60.4543       71
44872.192968 38299.220895           64           64.0 172.0000 166.1870      171
36163.280758 27454.368548          128        

In [34]:
%%time

! vw --final_regressor airbnb-lin-model-tokens.vw.bin airbnb-train-tokens.vw --passes 20 -l 3 -c -k --bit_precision 18

final_regressor = airbnb-lin-model-tokens.vw.bin
Num weight bits = 18
learning rate = 3
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airbnb-train-tokens.vw.cache
Reading datafile = airbnb-train-tokens.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
48400.000000 48400.000000            1            1.0 220.0000   0.0000       98
125101.125000 201802.250000            2            2.0 450.0000   0.7759        6
75713.196960 26325.268921            4            4.0 240.0000  15.5428       96
39055.715233 2398.233505            8            8.0  40.0000   4.1926       24
84181.565842 129307.416451           16           16.0 135.0000  38.5242      127
57145.070851 30108.575860           32           32.0  79.0000  30.2245       45
54811.209314 52477.347777           64           64.0 172.0000  81.9622      104
41783.796841 28756.384368          1

In [35]:
%%time

! vw --final_regressor airbnb-lin-model-stems.vw.bin airbnb-train-stems.vw --passes 20 -l 3 -c -k --bit_precision 18

final_regressor = airbnb-lin-model-stems.vw.bin
Num weight bits = 18
learning rate = 3
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airbnb-train-stems.vw.cache
Reading datafile = airbnb-train-stems.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
48400.000000 48400.000000            1            1.0 220.0000   0.0000       96
125062.570312 201725.140625            2            2.0 450.0000   0.8618        5
75542.389709 26022.209106            4            4.0 240.0000  16.8910       96
38900.226391 2258.063072            8            8.0  40.0000   5.2025       23
83627.623821 128355.021252           16           16.0 135.0000  47.9530      120
56023.923013 28420.222205           32           32.0  79.0000  30.9137       44
53072.210422 50120.497831           64           64.0 172.0000  88.2881      103
40484.896336 27897.582250          128 

Первый вывод, который можно сделать - даже на таком не очень большом датасете предобработка ускоряет обучение модели, в данном случае - просто за счет уменьшения количества признаков.

А что с качеством?

In [36]:
! vw --testonly --initial_regressor airbnb-lin-model-raw.vw.bin --predictions airbnb-1-predictions-raw.txt airbnb-test-raw.vw 


only testing
predictions = airbnb-1-predictions-raw.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test-raw.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
599.695374 599.695374            1            1.0  55.0000  79.4887       28
305.984709 12.274044            2            2.0  36.0000  39.5034      186
207.019877 108.055046            4            4.0  80.0000  67.7075       24
4919.375420 9631.730963            8            8.0  70.0000  74.8798      175
2999.275561 1079.175702           16           16.0 100.0000 142.4937      173
12161.945164 21324.614767           32           32.0 150.0000 152.4244      185
24285.223548 36408.501932           64           64.0 150.0000  82.4140       47
19419.673518 14554.123487          128          128.0 300.0000 122.5763       28
13868.897035 8318.120552     

In [37]:
! vw --testonly --initial_regressor airbnb-lin-model-tokens.vw.bin --predictions airbnb-1-predictions-tokens.txt airbnb-test-tokens.vw


only testing
predictions = airbnb-1-predictions-tokens.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test-tokens.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
852.703064 852.703064            1            1.0  55.0000  84.2011       19
433.786043 14.869022            2            2.0  36.0000  39.8560       92
280.920192 128.054341            4            4.0  80.0000  76.6952       17
5309.742549 10338.564906            8            8.0  70.0000  73.2144       95
3037.758406 765.774262           16           16.0 100.0000 129.0590      104
11758.061741 20478.365077           32           32.0 150.0000 135.2268       99
23336.385150 34914.708559           64           64.0 150.0000  91.4685       37
18445.038022 13553.690894          128          128.0 300.0000 136.3479       19
13362.039025 8279.04002

In [38]:
! vw --testonly --initial_regressor airbnb-lin-model-stems.vw.bin --predictions airbnb-1-predictions-stems.txt airbnb-test-stems.vw


only testing
predictions = airbnb-1-predictions-stems.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test-stems.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
980.918945 980.918945            1            1.0  55.0000  86.3196       19
565.562332 150.205719            2            2.0  36.0000  48.2558       88
630.141131 694.719930            4            4.0  80.0000  76.4350       17
5932.519242 11234.897353            8            8.0  70.0000  74.3693       90
3213.406160 494.293078           16           16.0 100.0000 109.6876      103
11970.408771 20727.411382           32           32.0 150.0000 140.3474       98
22687.461167 33404.513562           64           64.0 150.0000  82.0521       35
17803.776708 12920.092250          128          128.0 300.0000 138.9059       19
13497.156398 9190.536089

In [39]:
print("RAW")
print(calc_r2("airbnb-1-predictions-raw.txt", "airbnb-test-raw.vw"))

RAW
0.34760673021608524


In [40]:
print("TOKENS")
print(calc_r2("airbnb-1-predictions-tokens.txt", "airbnb-test-tokens.vw"))

TOKENS
0.3744525473300868


In [41]:
print("STEMS")
print(calc_r2("airbnb-1-predictions-stems.txt", "airbnb-test-stems.vw"))

STEMS
0.3578918225807295


Модели на сырых текстах и стеммах чуть хуже, чем модель на токенах без стоп-слов.

Как еще можно снизить числов признаков, оставив наиболее релевантные? Мы можем вычислить tf-idf веса для всех пар слово-документ в коллекции и оставить те слова, у которых веса больше некоторого порога.

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
X_train = X_raw
X_train.apply(lambda x: ' '.join(x))[0]

'hi everyone cosy bedroom in a modern apartment located in a central area paris th the apartment it s a bedrooms one is mine apartment of m sq ft fully renovated warm atmosphere a living room with a equiped kitchen wifi i provide towels and sheets central area cosmopolite non touristic very close to the marais bastille and republic transports metro walk saint ambroise line or richard lenoir line city bike station best j'

In [44]:
x_train, x_test = X_train_stems.apply(lambda x: ' '.join(x)) , X_test_stems.apply(lambda x: ' '.join(x)) 

Вычисляем tf-idf преобразование: в результате получится разреженная матрица

In [45]:
vec = TfidfVectorizer(ngram_range=(1, 1))
x_train_tfidf = vec.fit_transform(x_train)

x_train_tfidf

<54542x44692 sparse matrix of type '<class 'numpy.float64'>'
	with 3326139 stored elements in Compressed Sparse Row format>

Сделаем из этой матрицы список, в котором каждый элемент - это пара (w, tfidf(w, d , D)), где w - слово в документе d и коллекции документов D.

In [46]:
index_value={i[1]:i[0] for i in vec.vocabulary_.items()}

fully_indexed = []
for row in x_train_tfidf:
    fully_indexed.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

Посмотрим, например, на все слова с tf-idf >= 0.1 

In [47]:
tr = 0.1

[ (k, v ) for k, v in sorted(fully_indexed[4].items(), key=lambda item: -item[1]) if v >= tr]

[('doubl', 0.2580593795785133),
 ('split', 0.24115506933644754),
 ('neff', 0.2022642917696268),
 ('larg', 0.1962100846501572),
 ('origin', 0.18475126372913314),
 ('storag', 0.18029147151524685),
 ('level', 0.17138729784739948),
 ('slate', 0.16540308912231033),
 ('costa', 0.16540308912231033),
 ('side', 0.15770388173708214),
 ('finsburi', 0.1478633566100029),
 ('ceil', 0.14407523335957956),
 ('rais', 0.14128854967409324),
 ('front', 0.13858454237819792),
 ('place', 0.13625522822264793),
 ('form', 0.12490200014466063),
 ('convers', 0.12452005863135523),
 ('power', 0.12396028773002239),
 ('railway', 0.12042232913462157),
 ('chest', 0.11927026423601682),
 ('compris', 0.11767866051622303),
 ('benefit', 0.11727743203760296),
 ('shower', 0.11650003976394883),
 ('field', 0.11459355750386695),
 ('cupboard', 0.11364892399903367),
 ('follow', 0.11321064888943784),
 ('cooker', 0.11292363409760428),
 ('flat', 0.11097829368394094),
 ('hallway', 0.11085913741963234),
 ('leafi', 0.10875189729225716),


Попробуем в каждом документе оставлять топ-20 самых важных (с выскоим tf-idf) слов:

In [48]:
def important_words(x, n=20):
    return [(k.replace(' ', '_'), v) for k, v in sorted(x.items(), key=lambda item: -item[1]) if v >= tr][:n]

In [49]:
def convert_to_vw_tf(raw_text, target):
    return "{} |d {}".format(float(target), " ".join(["{}:{}".format(k, v) for k, v in raw_text]))

def write_vw_tf(X_data, Y_data, filename):
    with open(filename, "w") as f:
        for x, y in zip(X_data, Y_data):
            vw_object = convert_to_vw_tf(x, y)
            if not vw_object:
                continue
            f.write(vw_object + '\n')

In [50]:
x_train_updated = [important_words(x) for x in fully_indexed]
len(x_train_updated)

54542

In [51]:
x_train_updated[0]

[('suhd', 0.22702091541382582),
 ('leesa', 0.2092237299910606),
 ('olufsen', 0.20394496122668826),
 ('bialetti', 0.20394496122668826),
 ('sodastream', 0.19972994577223743),
 ('high', 0.19182189013874973),
 ('whitney', 0.18010511781362673),
 ('bang', 0.173690686601339),
 ('clutter', 0.16209468549638292),
 ('luxuri', 0.1596956274846224),
 ('asid', 0.1577930528524131),
 ('king', 0.15471383698293836),
 ('meatpack', 0.1502494251422974),
 ('samsung', 0.14901867589237205),
 ('apart', 0.14825207040563462),
 ('bluetooth', 0.1472195142269145),
 ('decid', 0.13671041244968138),
 ('renov', 0.13544658673657073),
 ('amazon', 0.1284808481645425),
 ('speaker', 0.12747745077125505)]

In [52]:
write_vw_tf(x_train_updated, y_train, "airbnb-train-important-stems.vw")

In [53]:
! head -n 3 airbnb-train-important-stems.vw

220.0 |d suhd:0.22702091541382582 leesa:0.2092237299910606 olufsen:0.20394496122668826 bialetti:0.20394496122668826 sodastream:0.19972994577223743 high:0.19182189013874973 whitney:0.18010511781362673 bang:0.173690686601339 clutter:0.16209468549638292 luxuri:0.1596956274846224 asid:0.1577930528524131 king:0.15471383698293836 meatpack:0.1502494251422974 samsung:0.14901867589237205 apart:0.14825207040563462 bluetooth:0.1472195142269145 decid:0.13671041244968138 renov:0.13544658673657073 amazon:0.1284808481645425 speaker:0.12747745077125505
450.0 |d gym:0.6752280868300573 amaz:0.5857861817339045 floor:0.34873046248046086 apart:0.2816179052965157
50.0 |d inst:0.3303694220060813 middl:0.2169097540580892 neiu:0.17606802050466333 argosi:0.17606802050466333 kendal:0.16518471100304066 depaul:0.16518471100304066 rican:0.16168106790911377 student:0.1615717403334601 intern:0.15009980649780472 peruvian:0.14667829926165915 dominican:0.14551470800700664 uic:0.14551470800700664 cultur:0.143791348024878

In [54]:
%%time

! vw --final_regressor airbnb-lin-model-tfidf.vw.bin airbnb-train-important-stems.vw --passes 15 -l 3 -c --bit_precision 18 -k

final_regressor = airbnb-lin-model-tfidf.vw.bin
Num weight bits = 18
learning rate = 3
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airbnb-train-important-stems.vw.cache
Reading datafile = airbnb-train-important-stems.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
48400.000000 48400.000000            1            1.0 220.0000   0.0000       21
124623.609375 200847.218750            2            2.0 450.0000   1.8402        5
76784.553589 28945.497803            4            4.0 240.0000   4.3585       21
40157.342880 3530.132172            8            8.0  40.0000   2.2763       16
87028.076649 133898.810417           16           16.0 135.0000   3.5806       21
63429.498878 39830.921108           32           32.0  79.0000   8.8392       21
65524.439860 67619.380842           64           64.0 172.0000  12.7161       21
53577.385478 41630.

Пока самое быстрое обучение модели (в предыдущие разы требовалось около 2.5-3.0 секунд). Можно попробовать обучать дольше, чтобы получить лучшее качество. Поднимем количество проходов по данным и посмотрим, какое качество сможем получить за то же время обучения.

In [55]:
%%time

! vw --final_regressor airbnb-lin-model-tfidf.vw.bin airbnb-train-important-stems.vw --passes 35 -l 3 -c --bit_precision 18 -k

final_regressor = airbnb-lin-model-tfidf.vw.bin
Num weight bits = 18
learning rate = 3
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = airbnb-train-important-stems.vw.cache
Reading datafile = airbnb-train-important-stems.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
48400.000000 48400.000000            1            1.0 220.0000   0.0000       21
124623.609375 200847.218750            2            2.0 450.0000   1.8402        5
76784.553589 28945.497803            4            4.0 240.0000   4.3585       21
40157.342880 3530.132172            8            8.0  40.0000   2.2763       16
87028.076649 133898.810417           16           16.0 135.0000   3.5806       21
63429.498878 39830.921108           32           32.0  79.0000   8.8392       21
65524.439860 67619.380842           64           64.0 172.0000  12.7161       21
53577.385478 41630.

In [56]:
x_test_tfidf = vec.transform(x_test)

fully_indexed_test = []
for row in x_test_tfidf:
    fully_indexed_test.append({index_value[column]:value for (column,value) in zip(row.indices,row.data)})

In [57]:
x_test_updated = [important_words(x) for x in fully_indexed_test]
write_vw_tf(x_test_updated, y_test, "airbnb-test-important-stems.vw")

In [58]:
! vw --testonly --initial_regressor airbnb-lin-model-tfidf.vw.bin --predictions airbnb-1-predictions-stems.txt airbnb-test-important-stems.vw

only testing
predictions = airbnb-1-predictions-stems.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = airbnb-test-important-stems.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
5731.760254 5731.760254            1            1.0  55.0000 130.7084       19
4014.961792 2298.163330            2            2.0  36.0000  83.9392       21
4030.666748 4046.371704            4            4.0  80.0000 121.7081       12
7675.492446 11320.318143            8            8.0  70.0000 108.3420       21
4766.873722 1858.254999           16           16.0 100.0000 153.5942       21
11999.351437 19231.829151           32           32.0 150.0000 145.4472       21
26360.865262 40722.379086           64           64.0 150.0000 175.8636       21
20796.356198 15231.847134          128          128.0 300.0000 284.8787       18
14396.3

In [59]:
print("TF-IDF")
print(calc_r2("airbnb-1-predictions-stems.txt", "airbnb-test-important-stems.vw"))

TF-IDF
0.3535020945957654


Видно, что качество сопоставимо с предыдущими запусками и лучше, чем в оригинальной модели, при этом сильно сжали данные - оставили всего по 20 слов в каждом тексте.

Конечно, различия о времени вычислений очень небольшие, и счет идет на секунды, однако, в реальной жизни масштаб будет значительно больше, и это будут часы или даже дни. Ожидается, что эффект от примененных приемов будет пропорциональным на реальных данных.

# Запускаем на кластере

In [60]:
import pyspark
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
    .setAppName("CourseraLocalSpark")
    .setMaster("local[*]")
)

sc = pyspark.SparkContext.getOrCreate(conf)
spark = SparkSession.builder.appName('NLP-examples').getOrCreate()

airbnb = spark.read.option("header",True).option("parserLib", "univocity")\
    .option("delimiter", "\t") \
    .csv('airbnb-100k.tsv')

In [61]:
airbnb.printSchema()

root
 |-- Price: string (nullable = true)
 |-- Description: string (nullable = true)



In [62]:
def all_preprocess(row):
    Y = row.Price
    X_raw = row.Description
    X_processed = stemm_description(preprocess(X_raw))
    return (Y, X_processed)

In [63]:
def convert_to_vw_spark(data):
    target, text = data
    return convert_to_vw(text, target)

In [64]:
X_rdd = (
    airbnb.rdd
    .filter(lambda x: bool(x.Price) and bool(x.Description))
    .filter(lambda x: detect_lang(x.Description)=='en')
    .map(all_preprocess)
    .cache()
)

In [66]:
%%time 

X_rdd.take(5)

CPU times: user 3.3 ms, sys: 589 µs, total: 3.89 ms
Wall time: 98.8 ms


[('50.0',
  ['everyon',
   'cosi',
   'bedroom',
   'modern',
   'apart',
   'locat',
   'central',
   'area',
   'pari',
   'apart',
   'bedroom',
   'one',
   'mine',
   'apart',
   'fulli',
   'renov',
   'warm',
   'atmospher',
   'live',
   'room',
   'equip',
   'kitchen',
   'wifi',
   'provid',
   'towel',
   'sheet',
   'central',
   'area',
   'cosmopolit',
   'non',
   'tourist',
   'close',
   'marai',
   'bastill',
   'republ',
   'transport',
   'metro',
   'walk',
   'saint',
   'ambrois',
   'line',
   'richard',
   'lenoir',
   'line',
   'citi',
   'bike',
   'station',
   'best']),
 ('125.0',
  ['comfort',
   'calm',
   'apart',
   'room',
   'center',
   'pari',
   'bastill',
   'area',
   'welcom',
   'explain',
   'live',
   'area',
   'bakeri',
   'market',
   'restaur',
   'bar',
   'live',
   'build',
   'close',
   'public',
   'transport',
   'metro',
   'ledru',
   'rollin',
   'charonn',
   'bastill',
   'gare',
   'lyon',
   'bus',
   'germain',
   'louvr'

In [64]:
print("\n".join(X_rdd.map(convert_to_vw_spark).take(5)))

50.0 |d everyon cosi bedroom modern apart locat central area pari apart bedroom one mine apart fulli renov warm atmospher live room equip kitchen wifi provid towel sheet central area cosmopolit non tourist close marai bastill republ transport metro walk saint ambrois line richard lenoir line citi bike station best
125.0 |d comfort calm apart room center pari bastill area welcom explain live area bakeri market restaur bar live build close public transport metro ledru rollin charonn bastill gare lyon bus germain louvr gare austerlitz gare nord live room bedroom kitchen bathroom sleep person doubl bed sofa bed also divid separ part quiet apart direct opposit love park central locat live restaur cafe fruit market live build love area nice vie quartier market bakeri apart equip fulli equip kitchen refriger freezer toaster kettl microwav induct hob dishwash wash machin coffe maker sheet towel linen provid broadband internet adsl
59.0 |d minut walk publiqu oberkampf nilmont lachais rent flat 

Для подсчета TF-IDF также можно воспользоваться стандартной библиотекой Spark. Для эффективной работы модуль расчета TF использует хеширование признаков.

In [65]:
from pyspark.mllib.feature import HashingTF, IDF

In [66]:
documents = X_rdd.map(lambda x: x[1])

hashingTF = HashingTF(numFeatures=2**14)
tf = hashingTF.transform(documents)
tf.cache()

PythonRDD[18] at RDD at PythonRDD.scala:53

In [67]:
%%time

tf.take(1)

CPU times: user 3.46 ms, sys: 650 µs, total: 4.11 ms
Wall time: 3.68 s


[SparseVector(16384, {436: 1.0, 812: 2.0, 1063: 1.0, 1202: 1.0, 1281: 1.0, 2081: 1.0, 2253: 1.0, 2374: 3.0, 2634: 1.0, 3420: 1.0, 3443: 1.0, 3613: 1.0, 3944: 1.0, 4750: 1.0, 4767: 1.0, 4949: 1.0, 5066: 1.0, 5789: 1.0, 6278: 1.0, 7254: 1.0, 7668: 1.0, 7695: 1.0, 8356: 1.0, 8459: 1.0, 8981: 2.0, 9012: 1.0, 9294: 1.0, 9536: 1.0, 11713: 1.0, 12131: 1.0, 12381: 1.0, 12556: 1.0, 12620: 1.0, 13067: 1.0, 13314: 1.0, 13467: 1.0, 13588: 1.0, 14296: 1.0, 14706: 2.0, 14722: 1.0, 15153: 1.0, 15706: 2.0})]

In [68]:
%%time

idf = IDF().fit(tf)
tfidf = idf.transform(tf)

CPU times: user 33 ms, sys: 13.8 ms, total: 46.7 ms
Wall time: 10min 35s


In [69]:
tfidf.take(1)

[SparseVector(16384, {436: 2.2216, 812: 3.4281, 1063: 5.5671, 1202: 7.5692, 1281: 2.1708, 2081: 4.5338, 2253: 2.9503, 2374: 2.2935, 2634: 3.9316, 3420: 2.1188, 3443: 0.6858, 3613: 0.608, 3944: 2.746, 4750: 1.1508, 4767: 1.1403, 4949: 2.804, 5066: 0.8053, 5789: 4.4441, 6278: 3.5627, 7254: 4.4749, 7668: 5.3961, 7695: 3.1881, 8356: 1.9695, 8459: 1.5621, 8981: 1.397, 9012: 3.2985, 9294: 1.4325, 9536: 8.3111, 11713: 3.1578, 12131: 1.7539, 12381: 7.1172, 12556: 1.8497, 12620: 0.6933, 13067: 2.14, 13314: 8.088, 13467: 2.033, 13588: 1.2257, 14296: 5.055, 14706: 2.1219, 14722: 0.5742, 15153: 1.5552, 15706: 4.0366})]